In [1]:
%%writefile app.py
import streamlit as st
from img_classification import teachable_machine_classification
import keras
from PIL import Image, ImageOps
import numpy as np

st.title("Sistema de Diagnóstico asistido por ordenador")
st.header("Clasificación de cáncer de piel mediante CNN")
st.text("Por favor introduce una imagen de una lesión cutánea que pueda sospechar DFSP, carcinoma o melanoma:")


@st.cache(allow_output_mutation=True)
def load_model():
    model = tf.keras.models.load_model("./model/model.h5")
    return model


with st.spinner('El modelo esta siendo cargado..'):
    model = load_model()

uploaded_file = st.file_uploader("Escoge una lesión cutánea ...", type="jpg")
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Imagen subida exitosamente.',
             use_column_width=True)
    st.write("")
    st.write("Clasificando...")
    prediction = teachable_machine_classification(image, "./model/model.h5")
    # score = tf.nn.softmax(predictions[0]) si no funca
    label = np.argmax(prediction)
    percentage = 100 * np.max(prediction)
    if label == 0:
        st.write(
            "La imagen es más probable que pertenezca a un Tumor benigno, con una probabilidad de: " + str(percentage))
    if label == 1:
        st.write("La imagen es más probable que pertenezca a un Carcinoma de celulas basales, con una probabilidad de: " + str(percentage))
    if label == 2:
        st.write(
            "La imagen es más probable que pertenezca a un DFSP, con una probabilidad de: " + str(percentage))
    if label == 3:
        st.write(
            "La imagen es más probable que pertenezca a un Melanoma, con una probabilidad de: " + str(percentage))

Overwriting app.py


In [2]:
%%writefile img_classification.py
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


def teachable_machine_classification(img, weights_file):
    # Cargar el modelo
    model = tensorflow.keras.models.load_model(weights_file)

    # crear una matriz del tamaño exacto para ser alimentado por el modelo keras
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = img
    # cambiando el tamaño de la imagen
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    # convirtiendo la imagen en un numpy array
    image_array = np.asarray(image)
    # normalizando la imagen
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Cargando la imagen en una matriz (paso que puede omitirse)
    data[0] = normalized_image_array

    # correr con el modelo
    prediction = model.predict(data)
    return (prediction)

Overwriting img_classification.py


In [3]:
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print('Link to web app:')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

Link to web app:
NgrokTunnel: "http://dd7b2b459f45.ngrok.io" -> "http://localhost:80"
2021-06-03 05:23:50.443 An update to the [server] config option section was detected. To have these changes be reflected, please restart streamlit.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/streamlit", line 8, in <module>
    sys.exit(main())
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.

t=2021-06-03T05:24:17+0000 lvl=warn msg="failed to open private leg" id=7ff875a7e107 privaddr=localhost:80 err="dial tcp 127.0.0.1:80: connect: connection refused"
t=2021-06-03T05:24:17+0000 lvl=warn msg="failed to open private leg" id=ed887e0d6aeb privaddr=localhost:80 err="dial tcp 127.0.0.1:80: connect: connection refused"
